#Preprocesamiento para AA
One hot encoding en el mes\
Imputación con un modelo para Entradas de personas por San sebastian (solo RG)\
Imputación de TOH y TOP para valores 2009 y 2010

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings

# Ignorar futuras advertencias de UserWarning para mantener la consola limpia
warnings.filterwarnings('ignore', category=UserWarning)

# Configurar display de pandas
pd.set_option('display.max_columns', None)

# 1. Cargar datos
try:
    df_rg = pd.read_csv('ExportedHojaRG.csv', encoding='latin-1')
except UnicodeDecodeError:
    print("Error al leer con 'latin-1', intentando con 'windows-1252'")
    df_rg = pd.read_csv('ExportedHojaRG.csv', encoding='windows-1252')

# Asegurarse de que 'Año' sea un tipo numérico (entero)
df_rg['Año'] = pd.to_numeric(df_rg['Año'], errors='coerce').fillna(0).astype(int)

# Manejo de la columna 'Mes':
df_rg['Mes'] = df_rg['Mes'].str.lower()

# Crear un mapeo de nombres de mes en español a números de mes
meses_a_numeros = {
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

# Añadir una columna numérica del mes (temporal para el proceso de imputación)
# Esto es más robusto que inferir del OHE para pd.to_datetime
df_rg['Mes_Num'] = df_rg['Mes'].map(meses_a_numeros)

# Realizar One-Hot Encoding para la columna 'Mes' (el original, en texto)
df_rg = pd.get_dummies(df_rg, columns=['Mes'], prefix='mes')


# 2. Tratamiento de tasas de ocupación (TOH/TOP)
def imputar_tasas_rg(df):
    print("--- Iniciando imputación de TOH/TOP ---")

    # Crear una columna de fecha temporal usando 'Año' y 'Mes_Num'
    # Esto es más robusto porque 'Mes_Num' es un entero y no depende de la interpretación de cadenas de mes
    # Asegurarse de que Año y Mes_Num sean numéricos antes de crear la fecha
    df['Fecha_temp'] = pd.to_datetime(df['Año'].astype(str) + '-' + df['Mes_Num'].astype(str) + '-01', errors='coerce')

    # Mensaje de depuración: Cuántas fechas temporales válidas se crearon
    print(f"Número de fechas temporales válidas creadas: {df['Fecha_temp'].notna().sum()} de {len(df)}")

    # Filtrar años con datos para referencia (2011 en adelante) y fechas válidas
    df_referencia = df[(df['Fecha_temp'].dt.year >= 2011) & (df['Fecha_temp'].notna())].copy()

    print(f"Filas de referencia (2011 en adelante con fecha válida): {len(df_referencia)}")

    if df_referencia.empty:
        print("Advertencia: No hay datos de referencia (2011 en adelante) para calcular la mediana mensual de TOH/TOP. No se realizará la imputación estacional.")
        df = df.drop(columns=['Fecha_temp', 'Mes_Num'], errors='ignore') # Limpiar columnas temporales
        return df

    # Calcular mediana mensual para las tasas de ocupación
    mediana_mensual = df_referencia.groupby(df_referencia['Fecha_temp'].dt.month)[
        ['Tasa de ocupación Habitaciones - TOH%', 'Tasa de ocupación Plazas - TOP%']
    ].median()

    # Mensaje de depuración: Mediana mensual calculada
    print("Mediana mensual calculada para TOH/TOP:\n", mediana_mensual)

    # Imputar valores faltantes para años anteriores a 2011
    for col in ['Tasa de ocupación Habitaciones - TOH%', 'Tasa de ocupación Plazas - TOP%']:
        mask_imputar = (df['Año'] < 2011) & (df[col].isna()) & (df['Fecha_temp'].notna())
        filas_imputadas_col = 0
        for idx in df[mask_imputar].index:
            mes_num = df.loc[idx, 'Fecha_temp'].month
            if mes_num in mediana_mensual.index:
                df.at[idx, col] = mediana_mensual.loc[mes_num, col]
                filas_imputadas_col += 1
            else:
                print(f"Advertencia: No se encontró mediana para el mes {mes_num} para la columna {col} en el año {df.loc[idx, 'Año']}.")
        print(f"Se imputaron {filas_imputadas_col} valores en la columna '{col}'.")

    # Eliminar las columnas temporales antes de devolver el DataFrame
    df = df.drop(columns=['Fecha_temp', 'Mes_Num'], errors='ignore')
    print("--- Imputación de TOH/TOP finalizada ---")
    return df

# Aplicar la imputación de tasas solo a df_rg
df_rg = imputar_tasas_rg(df_rg)

# 3. Modelo predictivo para entradas por San Sebastián
def entrenar_modelo_san_sebastian(df, ubicacion):
    # Preparamos una lista de las columnas dummy de mes
    mes_dummies_cols = [col for col in df.columns if col.startswith('mes_')]

    # Actualizamos la lista de features para el modelo
    features = [
        'Año',
        'Maxima media en C°', 'Minima media en C°', 'Media en C°', 'Lluvia caida en mm',
        'Dias con nieve', 'Desembarco Aeropuerto USH', 'Desembarco Aeropuerto RG'
    ] + mes_dummies_cols

    # Filtrar datos de entrenamiento (años 2011-2022)
    train = df[df['Año'] >= 2011].copy()

    existing_features = [f for f in features if f in train.columns]

    # Dropear NaNs solo en las columnas usadas para el entrenamiento (features y target)
    train = train.dropna(subset=existing_features + ['Entrada de personas por San Sebastian'])

    if train.empty:
        print(f"Advertencia: No hay datos suficientes para entrenar el modelo de San Sebastián para {ubicacion} (después de 2011 y eliminar NaNs).")
        return df

    X = train[existing_features]
    y = train['Entrada de personas por San Sebastian']

    modelo = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    modelo.fit(X, y)

    # Predecir valores faltantes (2009-2010)
    predict_data = df[(df['Año'] < 2011) &
                      (df['Entrada de personas por San Sebastian'].isna())].copy()

    if not predict_data.empty:
        # Imputar NaNs en X_pred con la media de las features del conjunto de entrenamiento
        X_pred = predict_data[existing_features].fillna(X.mean(numeric_only=True))

        predicciones = modelo.predict(X_pred)

        # Redondear las predicciones a números enteros
        predicciones_redondeadas = np.round(predicciones).astype(int)

        df.loc[predict_data.index, 'Entrada de personas por San Sebastian'] = predicciones_redondeadas

    # Validación del modelo
    if not X.empty and not y.empty:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        modelo.fit(X_train, y_train)
        preds = modelo.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, preds))

        # Calcular MAPE (manejar división por cero si y_test tiene ceros)
        # Usamos np.where para evitar la división por cero cuando y_test es 0
        mape_values = np.abs((y_test - preds) / y_test)
        mape_values = np.where(y_test == 0, 0, mape_values) # Si el valor real es 0, el error porcentual es 0.
        mape = np.mean(mape_values) * 100

        print(f'\n--- Validación del Modelo para {ubicacion} ---')
        print(f'RMSE: {rmse:.2f}')
        print(f'Media de los valores reales: {y_test.mean():.2f}')
        print(f'Desviación estándar de los valores reales: {y_test.std():.2f}')
        print(f'MAPE (Error Porcentual Absoluto Medio): {mape:.2f}%')
        print('-----------------------------------------')
    else:
        print(f"No hay suficientes datos para realizar la validación del modelo para {ubicacion}.")

    return df

# Aplicar el modelo predictivo solo a df_rg
df_rg = entrenar_modelo_san_sebastian(df_rg, 'Río Grande')

# 4. Guardar resultados
df_rg.to_csv('RG_completo.csv', index=False)

print("\nProcesamiento completado. Revisa 'RG_completo.csv'")

--- Iniciando imputación de TOH/TOP ---
Número de fechas temporales válidas creadas: 168 de 168
Filas de referencia (2011 en adelante con fecha válida): 144
Mediana mensual calculada para TOH/TOP:
             Tasa de ocupación Habitaciones - TOH%  \
Fecha_temp                                          
1                                           41.50   
2                                           43.95   
3                                           45.90   
4                                           48.65   
5                                           42.90   
6                                           36.60   
7                                           37.65   
8                                           40.40   
9                                           40.65   
10                                          44.25   
11                                          51.95   
12                                          35.35   

            Tasa de ocupación Plazas - TOP%  
Fecha_temp   